<a href="https://colab.research.google.com/github/Vaishnavit2000/airQualityAnalysis/blob/main/Copy_of_Air_Pollution_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anvil-uplink

     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 61kB 5.6MB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-cp36-none-any.whl size=45218 sha256=60c5578cc2fcf2dfd845fc024895c7b3106223d1a1408c8cffe90fddb4c553e0
  Stored in directory: /root/.cache/pip/wheels/a2/6e/4e/8b0ae12fb9b8a05715256952cf7609a8ab86285fab99b88c68
Successfully built ws4py


In [ ]:
import anvil.server

anvil.server.connect("LF7LHVR7OGKHIBQFAZ3C452O-TQQERCIGZZEJZEOM")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


**1. Plotted pie chart to show the major pollutant in pune city.**

In [ ]:
import anvil.mpl_util
import requests
import matplotlib.pyplot as plt
  
@anvil.server.callable
def make_plot(city):
  
  city=city.lower()
  url = 'http://api.waqi.info/feed/' + city + '/?token='
  api_key = '839dc94110707171ebdc025b28b20bd7d342aa56'

  #generated the url to access 
  main_url = url + api_key

  #request to the url
  r = requests.get(main_url)
  #print(r)
  data = r.json()['data']
  # data
  aqi = data['aqi']
  iaqi = data['iaqi']
  del iaqi['p']
  del iaqi['h']
  del iaqi['t']
  del iaqi['w']
  # print(iaqi)
  # for i in iaqi.items():
  #   print(i[0],':',i[1]['v'])
  
  pollutants = [i for i in iaqi]
  # print(pollutants)
  values = [i['v'] for i in iaqi.values()]
  # print(values)

  #plot a pie chart

  explode = [0 for i in pollutants]
  mx = values.index(max(values))
  # print(pollutants[mx])

  explode[mx] = 0.1
  plt.figure(figsize=(10,8))
  plt.pie(values , labels=pollutants , autopct='%1.1f%%' , shadow=True , explode=explode) 

  
  # Return this plot as a PNG image in a Media object
  return anvil.mpl_util.plot_image()

#Using Cartopy

In [ ]:
!apt-get install libproj-dev proj-data proj-bin
!apt-get install libgeos-dev
!pip install cython
!pip install cartopy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
proj-data is already the newest version (4.9.3-2).
proj-data set to manually installed.
The following NEW packages will be installed:
  libproj-dev proj-bin
0 upgraded, 2 newly installed, 0 to remove and 14 not upgraded.
Need to get 232 kB of archives.
After this operation, 1,220 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libproj-dev amd64 4.9.3-2 [199 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 proj-bin amd64 4.9.3-2 [32.3 kB]
Fetched 232 kB in 0s (2,444 kB/s)
Selecting previously unselected package libproj-dev:amd64.
(Reading database ... 144865 files and directories currently installed.)
Preparing to unpack .../libproj-dev_4.9.3-2_amd64.deb ...
Unpacking libproj-dev:amd64 (4.9.3-2) ...
Selecting previously unselected package proj-bin.
Preparing to unpack .../proj-bin_4.9.3-2_amd64.deb ...
Unpacking proj-bin (

In [ ]:
!pip uninstall shapely
!pip install shapely --no-binary shapely

Uninstalling Shapely-1.7.1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/Shapely-1.7.1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/shapely/*
Proceed (y/n)? y
  Successfully uninstalled Shapely-1.7.1
     |████████████████████████████████| 389kB 11.7MB/s 
Skipping wheel build for shapely, due to binaries being disabled for it.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
    Running setup.py install for shapely ... done


**2.Plotted the city on world map**

In [ ]:
import cartopy.crs as ccrs
import anvil.mpl_util
  
@anvil.server.callable
def show_on_map(city):
  url = 'http://api.waqi.info/feed/' + city + '/?token='
  api_key = '839dc94110707171ebdc025b28b20bd7d342aa56'

  #generated the url to access 
  main_url = url + api_key

  #request to the url
  r = requests.get(main_url)
  #print(r)
  data = r.json()['data']
  aqi = data['aqi']
  geo = data['city']['geo']
  fig = plt.figure(figsize=(10,8))
  ax = plt.axes(projection=ccrs.PlateCarree())
  ax.stock_img()

  plt.scatter(geo[1],geo[0],color='blue')
  plt.text(geo[1] + 3,geo[0]-2,f'{city} AQI \n   {aqi}',color='red')
  return anvil.mpl_util.plot_image()
  # plt.show()

#Analysis of Pune



**Plot comparision of pm25 before and after lockdown days** 

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import anvil.mpl_util
  
@anvil.server.callable
def com():
  csv_path = '/content/drive/MyDrive/Colab Notebooks/shivajinagar,-pune, pune, india-air-quality.csv'
  df = pd.read_csv(csv_path)

  df.columns
  df = df.rename(columns={' pm25':'PM25',
                          ' pm10':'PM10',
                          ' o3':'O3',
                          ' no2':'NO2',
                          ' co':'CO'   
  })

  df['date'] = pd.to_datetime(df.date)
  df21 = df.loc[df['date'].between( '2020-03-24', '2020-05-05' , inclusive=True)]
  df21 = df21.sort_values(by = 'date')
  df21.replace(' ' , '0' , inplace=True)
  pm25 = df21['PM25']
  pm25 = [int(i) for i in pm25]
  df_beflock = df.loc[df['date'].between( '2020-02-09', '2020-03-22' , inclusive=True)]
  df_beflock = df_beflock.sort_values(by = 'date')

  pm25_bef = df_beflock['PM25']
  pm25_bef = [int(i) for i in pm25_bef]
  plt.figure(figsize=(10,8))

  length = [i for i in range(1,len(df21)+1)]

  plt.plot(length,pm25,color='blue',label='under lockdown')
  plt.plot(length,pm25_bef,color='red',label='before lockdown')
  plt.legend()
  plt.title('Comparision of before lockdown vs under lockdown pm2.5 values')
  return anvil.mpl_util.plot_image()
# plt.show()

#Comparision of different cities

**Plotting bar graph for compsrision of various cities**

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

import anvil.mpl_util
  
@anvil.server.callable
def show_bar_chart():
  mumbai_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/bandra,-mumbai, india-air-quality.csv')
  # mumbai_data
  delhi_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sonia-vihar water treatment plant djb, delhi, delhi, india-air-quality.csv')
  # delhi_data
  kolkata_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kolkata-us consulate, india-air-quality.csv')

  hyderabad_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/central-university, hyderabad, india-air-quality.csv')
  # hyderabad_data
  pune_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/shivajinagar,-pune, pune, india-air-quality.csv')

  # rename_col()
  mumbai_data = mumbai_data.rename(columns={' pm25':'PM25',
                          ' pm10':'PM10',
                          ' o3':'O3',
                          ' no2':'NO2',
                          ' so2':'SO2',
                          ' co':'CO'   
  })


  delhi_data = delhi_data.rename(columns={' pm25':'PM25',
                          ' pm10':'PM10',
                          ' o3':'O3',
                          ' no2':'NO2',
                          ' so2':'SO2',
                          ' co':'CO'   
  })


  kolkata_data = kolkata_data.rename(columns={' pm25':'PM25',
                            
  })

  hyderabad_data = hyderabad_data.rename(columns={' pm25':'PM25',
                          ' pm10':'PM10',
                          ' o3':'O3',
                          ' no2':'NO2',
                          ' so2':'SO2',
                          ' co':'CO'   
  })

  pune_data = pune_data.rename(columns={' pm25':'PM25',
                          ' pm10':'PM10',
                          ' o3':'O3',
                          ' no2':'NO2',
                          ' co':'CO'   
  })

  # change_date()
  mumbai_data['date'] = pd.to_datetime(mumbai_data.date)
  delhi_data['date'] = pd.to_datetime(delhi_data.date)
  kolkata_data['date'] = pd.to_datetime(kolkata_data.date)
  hyderabad_data['date'] = pd.to_datetime(hyderabad_data.date)
  pune_data['date'] = pd.to_datetime(pune_data.date)

  # one_year()
  mum_one_year = mumbai_data.loc[mumbai_data['date'].between( '2019-01-01', '2020-01-01' , inclusive=True)]
  mum_one_year = mum_one_year.sort_values(by = 'date')

  delhi_one_year = delhi_data.loc[delhi_data['date'].between( '2019-01-01', '2020-01-01' , inclusive=True)]
  delhi_one_year = delhi_one_year.sort_values(by = 'date')

  kol_one_year = kolkata_data.loc[kolkata_data['date'].between( '2019-01-01', '2020-01-01' , inclusive=True)]
  kol_one_year = kol_one_year.sort_values(by = 'date')

  hyd_one_year = hyderabad_data.loc[hyderabad_data['date'].between( '2019-01-01', '2020-01-01' , inclusive=True)]
  hyd_one_year = hyd_one_year.sort_values(by = 'date')

  pune_one_year = pune_data.loc[pune_data['date'].between( '2019-01-01', '2020-01-01' , inclusive=True)]
  pune_one_year = pune_one_year.sort_values(by = 'date')

  # replace_zero()
  mum_one_year.replace(' ' , '0' , inplace=True)
  delhi_one_year.replace(' ' , '0' , inplace=True)
  kol_one_year.replace(' ' , '0' , inplace=True)
  hyd_one_year.replace(' ' , '0' , inplace=True)
  pune_one_year.replace(' ' , '0' , inplace=True)

  # average_all()
  mum_one_year['PM25'] = [int(i) for i in mum_one_year['PM25']]
  mum_avg = mum_one_year['PM25'].mean()

  delhi_one_year['PM25'] = [int(i) for i in delhi_one_year['PM25']]
  delhi_avg = delhi_one_year['PM25'].mean()

  kol_one_year['PM25'] = [int(i) for i in kol_one_year['PM25']]
  kol_avg = kol_one_year['PM25'].mean()

  hyd_one_year['PM25'] = [int(i) for i in hyd_one_year['PM25']]
  hyd_avg = hyd_one_year['PM25'].mean()

  pune_one_year['PM25'] = [int(i) for i in pune_one_year['PM25']]
  pune_avg = pune_one_year['PM25'].mean()
  
  x=['Pune' , 'Mumbai' , 'Delhi' , 'Kolkata' , 'Hyderabad']
  y=[pune_avg , mum_avg , delhi_avg , kol_avg , hyd_avg]
  plt.figure(figsize=(10,8))
  
  plt.xlabel('Cities')
  plt.ylabel("PM 2.5 concentration of a year")
  plt.title(" Bar Chart")
  plt.bar(x,y,label="PM 2.5",color='purple' )
  plt.legend()
  # plt.show()
  return anvil.mpl_util.plot_image()